# Aproximación a la puesta en producción de todo el workflow propuesto

Este notebook consiste en una aproximación a la puesta en producción end-to-end de todo el workflow propuesto. Incluye la introducción de los datos y la documentación por parte del cliente, la clasificación de los documentos, la extracción de los datos de los documentos, la validación de los datos, el almacenamiento de datos y reubicación de los documentos, el envío de los documentos al backoffice para su procesamiento manual y los avisos al cliente.

En el pdf `Workflow.pdf`, adjuntado en la documentación, se representa un diagrama con el con el workflow propuesto. El workflow end-to-end se divide en las siguientes fases principales:
 * Fase 0: Configuración del entorno
 * Fase 1: Aportación de los datos y de la documentación por parte del cliente
 * Fase 2: Validación formato, clasificación y reubicación de los documentos
 * Fase 3: Extracción, validación y almacenamiento de los datos y reubicación de los documentos
 * Fase 4: Monitorización y reajuste de los modelos

Para la extracción de datos en los documentos, ya se han demostrado las capacidades del modelo Donut DocVQA en la extranción de información a partir de preguntas en el notebook 3. En este ejemplo de puesta en producción, se le solicita al cliente que rellene su nombre y se contrarrestará con la documentación que ha adjuntado. Se considera que el emisor de los emails adjuntados corresponde al nombre del cliente. Este dato será el que se validará en la extración de información, una vez realizada la clasificación de los documentos.

# Fase 0: Configuración del entorno

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets sentencepiece

# Pdf to image
!pip install pdf2image
!sudo apt-get install poppler-utils
!pip install PyPDF2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 32.4 MB/s 
     |████████████████████████████████| 120 kB 65.0 MB/s 
     |████████████████████████████████| 365 kB 15.6 MB/s 
     |████████████████████████████████| 1.3 MB 72.6 MB/s 
     |████████████████████████████████| 212 kB 69.3 MB/s 
     |████████████████████████████████| 115 kB 69.6 MB/s 
     |████████████████████████████████| 127 kB 71.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to

In [ ]:
!rm -rf '/content/files_to_process'
!rm -rf '/content/output'
!rm -rf '/content/processed_files'
!rm -rf '/content/not_recognised_files'

In [ ]:
# create directory structure
import os

os.makedirs('/content/files_to_process')
os.makedirs('/content/output')
os.makedirs('/content/processed_files')
os.makedirs('/content/not_recognised_files')

classes=["ADVE", "email", "form", "memo", "news", "note", "receipt", "report", "resume", "passport"]
for i in classes:
  os.makedirs('/content/output/'+i)

## Fase 0: Aportación de los datos y de la documentación por parte del cliente

Se le solicita la cliente que rellene su nombre en la siguiente celda.

In [ ]:
name_real = input("Enter your full name (lastname, name): ") # Rubin, Kelly
print("Your full name is:", name_real)

Enter your full name (lastname, name): Rubin, Kelly
Your full name is: Rubin, Kelly


Se solicita al cliente que adjunte toda su documentación. Estos archivos se incluyen en la documentación.

In [ ]:
from google.colab import files

%cd content/files_to_process
files.upload()
%cd ..

/content/files_to_process


Saving DL.jpg to DL.jpg
/content


# Fase 1: Validación del formato, clasificación y reubicación de los documentos

## Cargar el model y processor

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("Mijavier/finetuned_donut_10_classes")
model = VisionEncoderDecoderModel.from_pretrained("Mijavier/finetuned_donut_10_classes")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/362 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
import torch

task_prompt = "<s_rvlcdip>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Importar las funciones necesarias:

In [ ]:
import re

def inference(image):
  # We prepare the image for the model using DonutProcessor
  pixel_values = processor(image, return_tensors="pt").pixel_values

  outputs = model.generate(pixel_values.to(device),
                               decoder_input_ids=decoder_input_ids.to(device),
                               max_length=model.decoder.config.max_position_embeddings,
                               early_stopping=True,
                               pad_token_id=processor.tokenizer.pad_token_id,
                               eos_token_id=processor.tokenizer.eos_token_id,
                               use_cache=True,
                               num_beams=1,
                               bad_words_ids=[[processor.tokenizer.unk_token_id]],
                               return_dict_in_generate=True,
                               output_scores=True,)
  return outputs

In [ ]:
def calculate_confidence(outputs):
  gen_sequences = outputs.sequences[:, decoder_input_ids.to(device).shape[-1]:-1]
  probs = torch.stack(outputs.scores, dim=1).softmax(-1)
  gen_probs = torch.gather(probs, 2, gen_sequences[:, :, None]).squeeze(-1)
  unique_prob_per_sequence = gen_probs.prod(-1)

  return unique_prob_per_sequence.item()

In [ ]:
def classification(outputs):
  seq = processor.batch_decode(outputs.sequences)[0]
  seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
  seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
  output = processor.token2json(seq)["class"] # convert to json and extract class

  return output

In [ ]:
import shutil

def file_organization(class_name, file_path):
  if class_name:
    shutil.copy(file_path, "output/"+class_name)
  else:
    shutil.copy(file_path, "not_recognised_files")
  shutil.move(file_path, "processed_files")

## Clasificación de los documentos

Se crea un dataframe donde almacenar la información de los documentos detectados.

In [ ]:
import pandas as pd

# create a dataframe for the appearance info
df = pd.DataFrame(columns=['class_name', 'num_documents'])
for class_name in classes:
  df.loc[len(df.index)] = [class_name, 0]

print(df)

  class_name num_documents
0       ADVE             0
1      email             0
2       form             0
3       memo             0
4       news             0
5       note             0
6    receipt             0
7     report             0
8     resume             0
9   passport             0


*Nota: Consultar el documento `Workflow.pdf` para facilitar la interpretabilidad de esta sección.*

Uno a uno, se procesan los documentos aportados. Los pdfs adjuntados se convierten en imágenes para su correcto procesado por los modelos Donut. En caso de que tenga más de una página, se creará una imagen por página y se clasificarán individualmente.

Si un documento no está en el formato soportado, se avisa a los clientes para que lo vuelva a aadjuntar en uno de los formatos soportados. Si un documento está en un formato adecuado. se hará la inferencia en el modelo de clasificación de Donut.

Se hace la inferencia de cada uno de los documentos aportados en formato correcto. Si un docuemento se considera no reconocido (por debajo del índice de confianza), se hace una copia del mismo y se almacenan en la carpeta `not_recognised_files` para que backoffice los clasifique manualmente. Con ello, también se abarca la casuística de que el cliente adjunte un documento incorrecto por error. Si está por encima del índice de confianza, se clasifica automáticamente. Se hace una copia del archivo y se reubica en la carpeta con su nombre de clase, dentro del directorio `output`.

En caso de que no se hayan detectado documentos de uno de las tipologías obligatorias, se avisa a los clientes con los documentos no detectados (faltaría considerar los clasificados manualmente por backoffice).

Por último, se reubican todos los documentos procesados al directorio `processed_files`. Habría que decidir si se mantienen o no estos documentos, ya que se ha reubicado una copia de cada uno de los archivos aportados con el formato adecuado, si bien, los pdfs han sido convertidos a imagen.

*NOTA: No se han modificado los nombres de los archivos al procesarlos y copiarlos a su nuevo directorio. Podría considerarse para una futura implementación.*



In [ ]:
import os
import glob
import numpy as np
import cv2
from pdf2image import convert_from_path
from pathlib import Path
from PyPDF2 import PdfFileWriter, PdfFileReader
import logging

logger = logging.getLogger("PyPDF2")
logger.setLevel(logging.ERROR)

# Load files
file_paths = glob.glob("files_to_process/*")

number_files = len(file_paths)
supported_files = [".jpg", ".jpeg", ".jpe", ".png", ".tif", ".tiff", ".jpf", ".j2c", ".j2k", ".jp2", ".jpc", ".jpx", ".bmp", ".dib", ".rle", ".jps", ".mpo", ".pbm", ".pam", ".pfm", ".pgm", ".pnm", ".ppm"]
j=1

for file_path in file_paths:
  # this will return a tuple of root and extension
  split_tup = os.path.splitext(file_path)

  file_name=Path(file_path).stem # file name
  file_extension = str(split_tup[1]).lower() # file extension
  file_name_extension=file_name+str(split_tup[1]) # file name and extension

  if j>1: print("-"*30)

  print("Document " + str(j) + "/" + str(number_files) + ": " + file_name_extension)

  if file_extension == ".pdf":
    pdf = PdfFileReader(open(file_path, "rb"))
    images = convert_from_path(file_path)

    for i in range(len(images)):
      # convert it from BGR to RGB channel
      images[i] = np.array(images[i])
      images[i] = np.array(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
      image = images[i]

      outputs = inference(image)
      confidence = calculate_confidence(outputs)

      output = PdfFileWriter()
      output.addPage(pdf.getPage(i))

      if confidence > 0.9999:
        class_name = classification(outputs)
        with open("output/"+class_name+"/"+file_name+"-page%s.pdf" % i, "wb") as outputStream:
            output.write(outputStream)

        print(" - Page " + str(i+1) + "/" + str(len(images)))
        print("   Class:", class_name)
        df.loc[classes.index(class_name)]["num_documents"] += 1
      else:
        print(" - ATTENTION! Page " + str(i+1) + "/" + str(len(images)) + " does not recognised.")
        print("   Send a notificacion to backoffice")
        with open("not_recognised_files/"+file_name+"-page%s.pdf" % i, "wb") as outputStream:
            output.write(outputStream)
    shutil.move(file_path, "processed_files")

  elif file_extension in supported_files:
    image = cv2.imread(file_path)

    outputs = inference(image)
    confidence = calculate_confidence(outputs)
    if confidence > 0.9999:
      class_name = classification(outputs)
      print("Class:", class_name)
      file_organization(class_name, file_path)
      df.loc[classes.index(class_name)]["num_documents"] += 1
    else:
      file_organization(None, file_path)
      print("ATTENTION! File not recognised.")
      print("Send a notificacion to backoffice.")

  else:
    print("ERROR! File extension", file_extension, "not supported")
    print("Send a notificacion to client: File extension not supported. Supported extensions are .pdf, .jpg, .jpeg, .jpe, .png, .tif, .tiff, .jpf, .j2c, .j2k, .jp2, .jpc, .jpx, .bmp, .dib, .rle, .jps, .mpo, .pbm, .pam, .pfm, .pgm, .pnm and .ppm. Please, import a supported file extension into the files_to_process folder and execute this cell again.")
    shutil.move(file_path, "processed_files")
  j+=1


print("\n NUMBER OF DOCUMENTS PER CLASS")
display(df)

undetected_files = []
for class_name in classes:
  num_documents = df.loc[classes.index(class_name)]["num_documents"]
  if num_documents == 0:
    undetected_files.append(class_name)

print("\n ERROR! Unrecognized files:")
for i in undetected_files:
  print(" - ", i)
print("Send a notificacion to client: Please, import this files into the files_to_process folder and execute this cell again.")
print("NOTE: Documents clasified by backoffice are not considered. Must be taken into account before notifying the client.")

Document 1/10: ADVE.jpg
ATTENTION! File not recognised.
Send a notificacion to backoffice.
------------------------------
Document 2/10: note.jpg
Class: note
------------------------------
Document 3/10: news.txt
ERROR! File extension .txt not supported
Send a notificacion to client: File extension not supported. Supported extensions are .pdf, .jpg, .jpeg, .jpe, .png, .tif, .tiff, .jpf, .j2c, .j2k, .jp2, .jpc, .jpx, .bmp, .dib, .rle, .jps, .mpo, .pbm, .pam, .pfm, .pgm, .pnm and .ppm. Please, import a supported file extension into the files_to_process folder and execute this cell again.
------------------------------
Document 4/10: form.jpg
Class: form
------------------------------
Document 5/10: receipt.pdf
 - Page 1/4
   Class: receipt
 - Page 2/4
   Class: receipt
 - Page 3/4
   Class: receipt
 - ATTENTION! Page 4/4 does not recognised.
   Send a notificacion to backoffice
------------------------------
Document 6/10: memo.jpg
Class: memo
------------------------------
Document 7/10

,class_name,num_documents
0,ADVE,0
1,email,1
2,form,1
3,memo,1
4,news,0
5,note,1
6,receipt,3
7,report,0
8,resume,1
9,passport,1



 ERROR! Unrecognized files:
 -  ADVE
 -  news
 -  report
Send a notificacion to client: Please, import this files into the files_to_process folder and execute this cell again.
NOTE: Documents clasified by backoffice are not considered. Must be taken into account before notifying the client.


# Fase 2: Extracción, validación y almacenamiento de los datos y reubicación de los documentos

Se extrae el nombre del emisor de los emails adjuntados por el cliente. Se considera que este nombre corresponde con el nombre del cliente, previamente facilitado como input por el cliente. Estos datos serán los que se validen posteriormente.

Para la extracción del nombre del emisor de los emails, se hace uso del modelo Donut DocVQA.

## Configuración del entorno

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
!pip install -q datasets sentencepiece

## Cargar imágenes

Se cargan las imáges que el modelo Donut anterior había clasificado como email.

Hay que tener en cuenta que los documentos que el cliente haya aportado en formato pdf, ya habrán sido procesados y convertidos a imagen en la fase de clasificación de los documentos, antes de hacer la inferencia sobre el modelo DocVQA.

In [ ]:
import glob

# Load files
file_paths = glob.glob("output/email/*")
file_paths.sort()

print(file_paths)

['output/email/email.jpg']


In [ ]:
from pathlib import Path

file_names = []
for file_path in file_paths:
  file_names.append(Path(file_path).name)

print(file_names)

['email.jpg']


In [ ]:
# Client data input
print("Client said his/her full name is:", name_real)
print("Let's probe it!")

Client said his/her full name is: Rubin, Kelly
Let's probe it!


Se crea un dataframe donde almacenaremos la información a evaluar.

In [ ]:
import pandas as pd

data = {'file_name': file_names, 'y_real': name_real.lower(), 'confidence':"", 'y_pred':"", 'eval_total':"", 'eval_partial':""}
df = pd.DataFrame.from_dict(data)

display(df)

,file_name,y_real,confidence,y_pred,eval_total,eval_partial
0,email.jpg,"rubin, kelly",,,,


## Cargar el model y processor

In [ ]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803M [00:00<?, ?B/s]

## Extracción del nombre emisor del email

Definimos la pregunta que queremos formular al modelo. En los emails, el nombre del emisor viene definido por la etiqueta From, así que la pregunta puede ser simplemente: From?

In [ ]:
question = "From?"

Se importa la función que calcula la confianza en las predicciones.

In [ ]:
def calculate_confidence(outputs, decoder_input_ids):
  gen_sequences = outputs.sequences[:, decoder_input_ids.to(device).shape[-1]:-1]
  probs = torch.stack(outputs.scores, dim=1).softmax(-1)
  gen_probs = torch.gather(probs, 2, gen_sequences[:, :, None]).squeeze(-1)
  unique_prob_per_sequence = gen_probs.prod(-1)

  return unique_prob_per_sequence.item()

Se extrae el nombre del emisor de los emails y se determina su grado de confianza en la predicción.

In [ ]:
import torch
import cv2
import re

for file_path in file_paths:
  image = cv2.imread(file_path)
  file_name = Path(file_path).name

  # We prepare the image for the model using DonutProcessor
  pixel_values = processor(image, return_tensors="pt").pixel_values

  task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
  prompt = task_prompt.replace("{user_input}", question)
  decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt")["input_ids"]

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)

  outputs = model.generate(pixel_values.to(device),
                                decoder_input_ids=decoder_input_ids.to(device),
                                max_length=model.decoder.config.max_position_embeddings,
                                early_stopping=True,
                                pad_token_id=processor.tokenizer.pad_token_id,
                                eos_token_id=processor.tokenizer.eos_token_id,
                                use_cache=True,
                                num_beams=1,
                                bad_words_ids=[[processor.tokenizer.unk_token_id]],
                                return_dict_in_generate=True,
                                output_scores=True)

  seq = processor.batch_decode(outputs.sequences)[0]
  seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
  seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token

  # confidance calculation
  confidence = calculate_confidence(outputs, decoder_input_ids)

  # We can convert the generated sequence to JSON ans extract the answer
  answer = processor.token2json(seq)["answer"]

  df.loc[df["file_name"] == file_name, 'y_pred'] = answer
  df.loc[df["file_name"] == file_name, 'confidence'] = confidence


Se calcula si la concordancia entre el nombre introducido por el cliente y el extraído por el modelo coinciden total o parcalmente. Se ignoran las diferencias entre mayúsculas y minúsculas. Por concordancia parcial se definen aquellas correlaciones de tres o más letras entre la clase real y la predicha o viceversa. Se muestran los resultados obtenidos.

In [ ]:
# calculation of total and partial match
df['eval_total'] = df.apply(lambda row: 1 if row.y_real ==  row.y_pred else 0, axis=1)
df['eval_partial'] = df.apply(lambda row: 1 if len(row.y_real)>2 and (row.y_pred in row.y_real or row.y_real in row.y_pred) else 0, axis=1)
display(df)

,file_name,y_real,confidence,y_pred,eval_total,eval_partial
0,email.jpg,"rubin, kelly",0.776337,"rubin, kelly",1,1


Importamos la función que copia los archivos al directorio deseado:

In [ ]:
import re

def copy_directory(path_to_copy, new_directory_name):
  name_directory = new_directory_name.replace(" ", "_")
  name_directory = re.sub(r'[^a-zA-Z0-9\._-]', '', name_directory)
  !ln -s $path_to_copy $name_directory

Si el resultado es mayor al índice de confianza definido, daremos por buena la predicción del modelo, si no, enviaremos el documento a backoffice para que extraiga la información.

En caso de que se de por buena la extracción de los datos:
* Si los datos coinciden totalmente: Se almacenan los datos y se se reubican los documentos en una nueva carpeta con el nombre del cliente (debería comprobarse que este nombre sea único)
* Si los datos no coinciden ni total ni parcialmente, se notifica al cliente para que adjunte la documentación correcta y/o modifique sus datos introducidos.
* Si los datos coinciden parcialmente pero no totalmente, decidir qué hacer.

In [ ]:
# Define a confidence index to balance between % of filtered data and accuracy
confidence_index = .5

# We select the document with the highest confidence
df.sort_values(by=['confidence'])
highest_confidence = df.iloc[0]['confidence']

if highest_confidence>confidence_index:
  name_pred = df.iloc[0]['y_pred']
  print("Name recognised:", name_pred)
  print("Name introduced manually:", name_real)

  if df.iloc[0]['eval_total'] == 1:
    print("Result: Total match")

    # Copy classified files to a new directory with extracted name. Note: Should be a unique value
    copy_directory("/content/output", name_pred)
    print("New directory created with validated customer documents.")

    print("Save validated data.")
    print("Send a notificacion to client: Data successfully validated!")
  elif df.iloc[0]['eval_partial'] == 1:
    print("Result: Partial match")
  else:
    print("Result: Not match")
    print("Send a message to client: Your name provided does not match with your document data. Please, try again uploading your document and/or introducing your name.")

else:
  print("Name not recognised.")
  print("Send a notificacion to backoffice.")


Name recognised: rubin, kelly
Name introduced manually: Rubin, Kelly
Result: Total match
New directory created with validated customer documents.
Save validated data.
Send a notificacion to client: Data successfully validated!


Este mismo ejercicio se podría realizar para cualquier documento y dato a extraer.

# Fase 4: Monitorización y reajuste de los modelos

Los modelos entrenados han sido monitorizados en Databricks con MLFlow. Gracias a ello, podemos observar el rendimiento de los entrenamientos de forma ordenada y clara. En la memoria se incluyen las métricas obtenidas.

Se podrían utilizar las herramientas que permite MLFlow para reentrenar los modelos en función del criterio estimado. Los modelos podrían ir perdiendo precisión con el paso delo tiempo. Es importante monitorizar el rendimiento de los modelos y reentrenar cuando sea necesario.

Respecto al modelo de clasificación, los documentos que han sido clasificados podrían servir para reentrenar al modelo. Esto es especialmente útil si se reentrena con los documentos clasificados por backoffice, que previamente habían tenido un grado de confianza bajo por el modelo. A su vez, habría que ir ajustando el índice de confianza del modelo para hacer un correcto equilibrio entre el porcentaje de los documentos automatizados y la precisión del modelo, en función de los KPIs definidos.

De la misma forma, un modelo preentrenado por nuestro sistema para la extracción de datos podría ir ajustándose con esta misma lógica.
